# Regression_v6 Trainingsskript

Mit diesem Jupyter Notebook Skript können Modelle für Spachtelmasse und Fugendeckstreifen trainiert werden. Das Skript ist für die Verwendung in Google Colab optimiert. Bilddaten und Labelinformationstabelle wurden im Vorfeld in Google Drive hochgeladen.

Mit dieser Zelle werden die notwendigen Pakete importiert.

In [2]:
import shutil
import matplotlib.pyplot as plt
from datetime import date, datetime
from random import shuffle

import csv
from csv import writer
import numpy as np
import os
import cv2
import tensorflow as tf
from keras.utils.layer_utils import count_params
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten
from keras import backend as K 
from sklearn.model_selection import train_test_split

Diese Zelle importiert die verwendeten Methoden. Es sind mehrere Hilfsmethoden sowie die Hauptmethode *doTrainingwithParameters* an welche die Trainingsparameter übergeben werden können. Die Funktion der Methoden ist in den Kommentaren erklärt. Für ein Training ist keine Anpassung in der Zelle erforderlich. Ein Training kann über die nachfolgenden Zellen erfolgen.

In [3]:
from matplotlib.bezier import split_bezier_intersecting_with_closedpath
##################################################################################
# Methoden (keine Anpassung notwendig!)
##################################################################################

##################################################################################
# Methoden zum überprüfen der Bilder (Keine Anpassung notwendig)
##################################################################################

def check_synthetic_mirror(splittedfilename):
    if (splittedfilename[4] == str(2)):
      return True
    return False  

def check_synthetic_rotation(splittedfilename):
    if (splittedfilename[5] == str(2)):
      return True
    return False  

def check_synthetic_noise(splittedfilename):
    if (splittedfilename[6] == str(2)):
      return True
    return False  

def check_synthetic_brightness(splittedfilename):
    if (splittedfilename[8] == str(2) or splittedfilename[8] == str(3)):
      return True
    return False  

def check_synthetic_sharpness(splittedfilename):
    if (splittedfilename[10] == str(2) or splittedfilename[10] == str(3)):
      return True
    return False  

def check_images_with_sticker(splittedfilename):
    if (splittedfilename[15] == str(2) or splittedfilename[15] == str(3) or splittedfilename[15] == str(4)):
      return True
    return False  

def check_images_taken_with_DSLR(splittedfilename):
    if (splittedfilename[13] == str(2)):
      return True
    return False  

def check_images_taken_with_Smartphone(splittedfilename):
    if (splittedfilename[13] == str(1)):
      return True
    return False  

def check_images_taken_format_landscape(splittedfilename):
    if (splittedfilename[14] == str(1)):
      return True
    return False  

def check_images_taken_format_portrait(splittedfilename):
    if (splittedfilename[14] == str(2)):
      return True
    return False  

def check_ImageIsOriginal(splittedfilename):
    if ((splittedfilename[4] == str(1)) and
    (splittedfilename[5] == str(1)) and
    (splittedfilename[6] == str(1)) and
    (splittedfilename[7] == str(1)) and
    (splittedfilename[8] == str(1)) and
    (splittedfilename[9] == str(1)) and
    (splittedfilename[10] == str(1)) and
    (splittedfilename[11] == str(1)) and
    (splittedfilename[12] == str(1)) and
    (splittedfilename[15] == str(1))
    ):
      return True
    return False  


##################################################################################
# Methode zum überprüfen welche Bilder verwendet werden sollen (keine Anpassung notwendig!)
##################################################################################
def check_ImageIsInFilter(splittedfilename, images_with_sticker, images_taken_with_DSLR, images_taken_with_Smartphone, 
                    images_taken_format_landscape, images_taken_format_portrait, synthetic_images_mirror, 
                    synthetic_images_rotation, synthetic_images_noise, synthetic_images_brightness, synthetic_images_sharpness):

    synthetic_checked_mirror = check_synthetic_mirror(splittedfilename)
    synthetic_checked_rotation = check_synthetic_rotation(splittedfilename)
    synthetic_checked_noise = check_synthetic_noise(splittedfilename)
    synthetic_checked_brightness = check_synthetic_brightness(splittedfilename)
    synthetic_checked_sharpness = check_synthetic_sharpness(splittedfilename)
    images_with_sticker_checked = check_images_with_sticker(splittedfilename)
    images_taken_with_DSLR_checked = check_images_taken_with_DSLR(splittedfilename)
    images_taken_with_Smartphone_checked = check_images_taken_with_Smartphone(splittedfilename)
    images_taken_format_landscape_checked = check_images_taken_format_landscape(splittedfilename)
    images_taken_format_portrait_checked = check_images_taken_format_portrait(splittedfilename)

    if (synthetic_images_mirror == 0 and synthetic_checked_mirror == True):
        return False
    if (synthetic_images_rotation == 0 and synthetic_checked_rotation == True):
        return False
    if (synthetic_images_noise == 0 and synthetic_checked_noise == True):
        return False
    if (synthetic_images_brightness == 0 and synthetic_checked_brightness == True):
        return False
    if (synthetic_images_sharpness == 0 and synthetic_checked_sharpness == True):
        return False
    if (images_with_sticker == 0 and images_with_sticker_checked == True):
        return False
    if (images_taken_with_DSLR == 0 and images_taken_with_DSLR_checked == True):
        return False
    if (images_taken_with_Smartphone == 0 and images_taken_with_Smartphone_checked == True):
        return False
    if (images_taken_format_landscape == 0 and images_taken_format_landscape_checked == True):
        return False
    if (images_taken_format_portrait == 0 and images_taken_format_portrait_checked == True):
        return False
    return True



##################################################################################
# Methoden zum Anlegen der Ordner (keine Anpassung notwendig!)
##################################################################################
def createDirDeleteOldDir(path):
    if os.path.exists(path):
        shutil.rmtree(path, ignore_errors=False)
    os.makedirs(path)

def createDirIfNotExists(path):
    if not os.path.exists(path):
        os.makedirs(path)





def doTrainingwithParameters(srcpath_images, srcpath_label_infos, csv_name, csv_spalte_labelinfo, destpath_dir, evalpath_output, model_number, 
                        epochs_train, training_number, images_with_sticker, images_taken_with_DSLR, images_taken_with_Smartphone, 
                        images_taken_format_landscape, images_taken_format_portrait, synthetic_images_mirror, 
                        synthetic_images_rotation, synthetic_images_noise, synthetic_images_brightness, synthetic_images_sharpness, 
                        saving_images_to_dir, saving_model_to_dir, normalizing_images, standardizing_images, preprocesing_images, resizing_images, widthquer, heightquer, faktorTrainSplit):
 
    #Delete Model / Clear session
    K.clear_session()

    ####################################################################
    # Datum und Uhrzeit des Trainings
    ####################################################################
    today = date.today()
    now = datetime.now()
    # dd/mm/yyyy
    date_of_training = today.strftime("%d/%m/%Y")
    time_of_training = now.strftime("%H:%M:%S")

    ###############################################
    # Gibt den Zielpfad für Infos des Trainings an
    ###############################################
    destpath_batchplots = destpath_dir + "Plots/"
    destpath_dir = destpath_dir + "Training" + str(training_number) + "/"
    # Gibt die Pfade zum Zwischenspeichern der verwendeten Bilder an
    # VORSICHT! ORDNER WERDEN BEIM START DES SKRIPTS GELÖSCHT/GELEERT!
    destpath_train_images = destpath_dir + "Dataset/Trainingset/"
    destpath_val_images = destpath_dir + "Dataset/Validationset/"
    # Gibt den Pfad zum speichern des trainierten Modells an
    destpath_modelsave = destpath_dir + "models/"
    # Gibt den Pfad zum Speichern der Modell-Plots an
    destpath_plotsave = destpath_dir + "Plot/"
    destpath_logsave = destpath_dir + "Logs/"


    ###################################################################
    # Einlesen der Daten (Bilder und Label) in Arrays
    ###################################################################

    # Hilfslisten und Variablen zur Verteilung des Train-/Testsplits
    waende_alle = []
    waende_train = []
    waende_val = []
    # Arrays für die Images des Train- und Validationsplits
    x_train_material = []
    x_val_material = []
    # Array für die Labels des Train- und Validationsplits
    y_train_material = []
    y_val_material = []

    # Hilfsarray um die Dateinamen aller verwendeten Bilder zwischenzuspeichern
    images_train_filenames = []
    images_val_filenames = []
    images_all_filenames = []

    # Legt die Ordner Struktur an
    createDirIfNotExists(destpath_batchplots)
    createDirDeleteOldDir(destpath_dir)
    if saving_images_to_dir == 1:
      createDirDeleteOldDir(destpath_train_images)
      createDirDeleteOldDir(destpath_val_images)
    createDirDeleteOldDir(destpath_modelsave)
    createDirDeleteOldDir(destpath_logsave)
    createDirDeleteOldDir(destpath_plotsave)


    ##################################################################################
    # Wand-Label-Info-CSV in NP Array einlesen
    ##################################################################################
    with open(str(srcpath_label_infos)+str(csv_name)) as wand_label_info_file:
        wand_label_info = np.loadtxt(wand_label_info_file, delimiter=";", skiprows=1, dtype="int")


    ##################################################################################
    # Ausgabe Parameter und Infos auf der Konsole
    ##################################################################################
    print()
    print("############################################################################################")
    print("#                                                                                          #")
    print("#                                  Training "+str(training_number)+"                                             #")
    print("#                                                                                          #")
    print("############################################################################################")
    print()

    print("Parameter:")
    print("##########")
    print()
    print("Modellnummer: "+str(model_number))
    print("Normalisierung: "+str(normalizing_images))
    print("Standardisierung: "+str(standardizing_images))
    print("Preprocessing: "+str(preprocesing_images))
    print("Bilder mit Sticker (rot, Aruco, etc.): "+str(images_with_sticker))
    print("Auflösung Breite: "+str(widthquer))
    print("Auflösung Höhe: "+str(heightquer))
    print("Synthetisch - gespiegelt: "+str(synthetic_images_mirror))
    print("Synthetisch - rotiert: "+str(synthetic_images_rotation))
    print("Synthetisch - Rauschen: "+str(synthetic_images_noise))
    print("Synthetisch - Helligkeit: "+str(synthetic_images_brightness))
    print("Synthetisch - Schärfe: "+str(synthetic_images_sharpness))
    print()


    ##################################################################################
    # Baustellen und Wand ID aus Dateinamen laden um Train/Testsplit auf Wandebene vorzubereiten
    ##################################################################################
    for filename in os.listdir(srcpath_images):
        # Trennt den Filename in den reinen Filename und Extension
        filenameshort, extension = os.path.splitext(filename)
        # Prüft ob Datei .jpg ist
        if extension == ".jpg":
            # Splitted den Dateinamen mit dem Trennoperator "_" und speichert den Inhalt in eine Liste
            splittedfilename = filenameshort.split("_")
            # Speichert die Baustellennummer und Wandnummer zwischen
            img_baustellenID = splittedfilename[1]
            img_wandID = splittedfilename[2]
            img_baustellenUndWandID = img_baustellenID+"_"+img_wandID
            # Welche Images sollen aus dem vorher angegebenen SrcOrdner verwendet werden. Gefiltert wird nach den Flags im Dateinamen
            if (check_ImageIsInFilter(splittedfilename, images_with_sticker, images_taken_with_DSLR, images_taken_with_Smartphone, 
                    images_taken_format_landscape, images_taken_format_portrait, synthetic_images_mirror, 
                    synthetic_images_rotation, synthetic_images_noise, synthetic_images_brightness, synthetic_images_sharpness)):
                if img_baustellenUndWandID not in waende_alle:
                    waende_alle.append(img_baustellenUndWandID)
    waende_alle_count = len(waende_alle)  
    print()
    print("Einlesen der Daten:")
    print("###################")
    print()        
    print("Anzahl aller verwendeten Wände/Szenen: "+str(waende_alle_count))
    # Bestimmt die Anzahl der Wände für den Trainingssplit. Dafür wird der vorherig definierte Faktor verwendet
    waende_train, waende_val = train_test_split(waende_alle, train_size=faktorTrainSplit, random_state=42)
    waende_train_count = len(waende_train)
    print("Anzahl der Wände/Szenen für das Trainingset: "+str(waende_train_count))
    waende_val_count = len(waende_val)
    print("Anzahl der Wände/Szenen für das Validationset: "+str(waende_val_count))


    ##################################################################################
    # gewünschte Bilder werden bearbeitet und dem Train- oder Testsplit hinzugefügt
    ##################################################################################
    for filename in os.listdir(srcpath_images):
        # Trennt den Filename in den reinen Filename und Extension
        filenameshort, extension = os.path.splitext(filename)
        # Prüft ob Datei .jpg ist
        if extension == ".jpg":
            # Splitted den Dateinamen mit dem Trennoperator "_" und speichert den Inhalt in eine Liste
            splittedfilename = filenameshort.split("_")
            # Speichert die Baustellennummer und Wandnummer zwischen
            img_baustellenID = splittedfilename[1]
            img_wandID = splittedfilename[2]
            img_baustellenUndWandID = img_baustellenID+"_"+img_wandID
            # Welche Images sollen aus dem vorher angegebenen Src-Ordner verwendet werden. 
            # Gefiltert wird nach den Flags im Dateinamen (siehe Methode "check_ImageIsInFilter")
            if (check_ImageIsInFilter(splittedfilename, images_with_sticker, images_taken_with_DSLR, images_taken_with_Smartphone, 
                    images_taken_format_landscape, images_taken_format_portrait, synthetic_images_mirror, 
                    synthetic_images_rotation, synthetic_images_noise, synthetic_images_brightness, synthetic_images_sharpness)):
                # Gefilterte Bilder/Images laden
                #print(filename)
                img = cv2.imread(os.path.join(srcpath_images,filename))


                ###################################################################################
                # Preprocessing1 der Bilder
                ###################################################################################
                if preprocesing_images == 1:
                  imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #Bild wird schwarz weiß
                  # print(f"Shape imgGray: {imgGray.shape}")
                  imgBlur=cv2.GaussianBlur(imgGray,(5,5),1) #Kernel 5*5
                  # print(f"Shape imgBlur: {imgGray.shape}")
                  # Canny Filter: Thr = For this, we need two threshold values, minVal and maxVal. Any edges with intensity gradient more than maxVal are sure to be edges and those below minVal are sure to be non-edges, so discarded.
                  imgCanny=cv2.Canny(imgBlur, 40, 40) #Cannyfilter (25, 25 guter Wert, wenn nur Canny)
                  # print(f"Shape imgCanny: {imgCanny.shape}")
                  kernel= np.ones((5,5)) #Neuer Kernel für Dilate (5*5 groß)
                  imgDial = cv2.dilate(imgCanny,kernel,iterations=2) #Erweitert das Bild (verstärkt)
                  # print(f"Shape imgDial: {imgDial.shape}")
                  imgThre=cv2.erode(imgDial,kernel,iterations=1) #Verfeinert das Bild (verwässert)
                  # print(f"Shape imgThre: {imgThre.shape}")
                  img = imgThre


                ##################################################################################
                # Resizing der Bilder
                ##################################################################################
                if resizing_images == 1:
                  # Resize der Bilder auf neue Maße wie zu Beginn angegeben
                  # Prüft ob Bild Querformat ist
                  if check_images_taken_format_landscape(splittedfilename):
                      dim = (widthquer, heightquer)
                  # Prüft ob Bild Hochformat ist
                  elif check_images_taken_format_portrait(splittedfilename):
                      dim = (widthhoch, heighthoch)
                  # Resize mit neuen Dimensionen
                  resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
                  img = resized


                ###################################################################################
                # Speichern der Bilder in die zuvor angegebenen Zielordner  
                ###################################################################################
                if saving_images_to_dir == 1:
                  if img_baustellenUndWandID in waende_train:
                      cv2.imwrite(os.path.join(destpath_train_images , filenameshort+'.jpg'),img)
                  elif check_ImageIsOriginal(splittedfilename):
                      cv2.imwrite(os.path.join(destpath_val_images , filenameshort+'.jpg'),img)


                ###################################################################################
                # Normalisierung
                ###################################################################################
                if normalizing_images == 1:
                  img = img/255


                ###################################################################################
                # Standardisierung
                ###################################################################################
                if standardizing_images == 1:
                  img = np.array(img, dtype=np.int64)
                  imgst = (img - np.mean(img)) / np.std(img)
                  img = imgst


                ###################################################################################
                # Preprocessing2 der Bilder 
                ###################################################################################
                # (S/W Bilder haben weniger Dimensionen als Bilder in Farbe, weshalb weitere hinzugefügt werden)
                if preprocesing_images == 1:
                  # print(f"Shape before new dimension: {img.shape}")
                  img = tf.expand_dims(img,axis=2)
                  # print(f"Shape after new dimension: {img.shape}")


                ##################################################################################
                # Speichern der Bilder und der zugehörigen Labels in die Arrays des Train- und Testsets  
                ################################################################################## 
                # Prüft ob das Bild zu den Traingswänden gehört
                if img_baustellenUndWandID in waende_train:
                    x_train_material.append(np.array(img))
                    # Iteriert durch die Infos aus der CSV und schaut ob die BaustellenID und WandID der Dateien mit den Zeilen in der CSV matchen. Falls ja, dann soll das zugehörige 
                    # Label in das Array "y_train_material" gespeichert werden. 
                    for i in range(len(wand_label_info)):
                        # Prüft ob BaustellenID und WandID mit den Spalten in der CSV übereinstimmen.
                        if (
                            str(wand_label_info[i][0]) == img_baustellenID and
                            str(wand_label_info[i][1]) == img_wandID
                        ):
                            # Fügt den Wert der gewählten Spalte in der CSV dem Array "y_train_material" hinzu
                            y_train_material.append(np.array(wand_label_info[i][csv_spalte_labelinfo]))
                    # Den Dateinamen des gefilterten Traings-Images in einem Array speichern. Für spätere Kontrolle nützlich
                    images_train_filenames.append(filename)
                    # Den Dateinamen des gefilterten Images in einem Array speichern in dem alle verwendeten Dateinamen gespeichert werden. Für spätere Kontrolle nützlich
                    images_all_filenames.append(filename)
                # Prüft ob das Bild zu den Validationwänden gehört, dabei werden nur Originalbilder dem Validationsplit hinzugefügt
                elif check_ImageIsOriginal(splittedfilename):
                    x_val_material.append(np.array(img))
                    # Iteriert durch die Infos aus der CSV und schaut ob die BaustellenID und WandID der Dateien mit den Zeilen in der CSV matchen. Falls ja, dann soll das zugehörige 
                    # Label in das Array "y_val_material" gespeichert werden. 
                    for i in range(len(wand_label_info)):
                        # Prüft ob BaustellenID und WandID mit den Spalten in der CSV übereinstimmen.
                        if (
                            str(wand_label_info[i][0]) == img_baustellenID and
                            str(wand_label_info[i][1]) == img_wandID
                        ):
                            # Fügt den Wert der gewählten Spalte in der CSV dem Array "y_train_material" hinzu
                            y_val_material.append(np.array(wand_label_info[i][csv_spalte_labelinfo]))
                    # Den Dateinamen des gefilterten Traings-Images in einem Array speichern. Für spätere Kontrolle nützlich
                    images_val_filenames.append(filename)
                    # Den Dateinamen des gefilterten Images in einem Array speichern in dem alle verwendeten Dateinamen gespeichert werden. Für spätere Kontrolle nützlich
                    images_all_filenames.append(filename)


    ##################################################################################
    # Konvertierung in NP Arrays
    ################################################################################## 
    images_train_filename = np.array(images_train_filenames)
    images_val_filename = np.array(images_val_filenames)
    images_all_filenames = np.array(images_all_filenames)
    x_train_material = np.array(x_train_material)
    y_train_material = np.array(y_train_material)
    x_val_material = np.array(x_val_material)
    y_val_material = np.array(y_val_material)


    ##################################################################################
    # Ausgabe der Werte auf der Konsole
    ################################################################################## 
    print()
    images_all_count = len(images_all_filenames)
    print("Anzahl aller verwendeter Bilder: "+str(images_all_count))
    print()

    print("Traingset:")
    x_train_images_count = len(x_train_material)
    print("Anzahl der im Array x_train_material gespeicherten Bilder: "+str(x_train_images_count))
    y_train_material_labelcount = len(y_train_material)
    print("Anzahl der im Array y_train_material gespeicherten Labels: "+str(y_train_material_labelcount))
    y_train_material_min = np.amin(y_train_material)
    print("y_train_material Min: "+str(y_train_material_min))
    y_train_material_max = np.amax(y_train_material)
    print("y_train_material Max: "+str(y_train_material_max))
    y_train_material_mean = round(np.mean(y_train_material))
    print("y_train_material Mean: "+str(y_train_material_mean))
    print()

    print("Validationset:")
    x_val_images_count = len(x_val_material)
    print("Anzahl der im Array x_val_material gespeicherten Bilder: "+str(x_val_images_count))
    y_val_material_labelcount = len(y_val_material)
    print("Anzahl der im Array y_val_material gespeicherten Labels: "+str(y_val_material_labelcount))
    y_val_material_min = np.amin(y_val_material)
    print("y_val_material Min: "+str(y_val_material_min))
    y_val_material_max = np.amax(y_val_material)
    print("y_val_material Max: "+str(y_val_material_max))
    y_val_material_mean = round(np.mean(y_val_material))
    print("y_val_material Mean: "+str(y_val_material_mean))
    print()


    ##################################################################################
    # Definieren des Modells
    ##################################################################################  

    if preprocesing_images == 1:
      # Wenn Preproccessng genutzt wird ist das Bild schwarz/weiß. Dadurch hat das Bild nur eine Tiefe von einer Schicht.
      image_layer = 1
    else:
      # Falls kein Preproccessing genutzt wird ist das Bild in Farbe. Dadurch hat das Bild eine Tiefe von drei Schichten.
      image_layer = 3

    print()
    print("Definieren des Modells:")
    print("#######################")
    print()
    # Welches Modell gewählt wird, wird über einen Trainingsparameter gesteuert.
    if model_number == 1:
        print("Modell 1 gewählt")
        tf.keras.backend.clear_session()
        material_model = Sequential([
            Conv2D(128, kernel_size=3, activation='relu', input_shape=(heightquer, widthquer, image_layer)),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(128, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(256, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D (512, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Flatten(),
            Dropout(0.5),
            Dense(512, activation='relu'),
            Dense(1, activation='linear', name='material')
        ])
    elif model_number == 2:
        print("Modell 2 gewählt")
        tf.keras.backend.clear_session()
        material_model = Sequential([
            Conv2D(128, kernel_size=3, activation='relu', input_shape=(heightquer, widthquer, image_layer)),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(256, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(512, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(1024, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Flatten(),
            Dropout(0.5),
            Dense(1024, activation='relu'),
            Dense(1, activation='linear', name='material')
        ])
    elif model_number == 3:
        print("Modell 3 gewählt")
        tf.keras.backend.clear_session()
        material_model = Sequential([
            Conv2D(128, kernel_size=3, activation='relu', input_shape=(heightquer, widthquer, image_layer)),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(128, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(256, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(256, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(512, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Flatten(),
            Dropout(0.5),
            Dense(512, activation='relu'),
            Dense(1, activation='linear', name='material')
        ])
    elif model_number == 4:
        print("Modell 4 gewählt")
        tf.keras.backend.clear_session()
        material_model = Sequential([
            Conv2D(128, kernel_size=3, activation='relu', input_shape=(heightquer, widthquer, image_layer)),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(128, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(256, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(512, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(1024, kernel_size=3, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Flatten(),
            Dropout(0.5),
            Dense(1024, activation='relu'),
            Dense(1, activation='linear', name='material')
        ])
    else:
        print("Modell 5 gewählt")
        tf.keras.backend.clear_session()
        material_model = Sequential([
            Conv2D(128, kernel_size=5, strides=2, activation='relu', input_shape=(heightquer, widthquer, image_layer)),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(256, kernel_size=5, strides=2, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Conv2D(512, kernel_size=3, strides=2, activation='relu'),
            MaxPool2D(pool_size=3, strides=2),
            Flatten(),
            Dropout(0.5),
            Dense(512, activation='relu'),
            Dense(1, activation='linear', name='material')
        ])

    material_model.compile(
    optimizer='adam',
    loss='mse',
    metrics =['mae']
    )

    # Legt den CSV Callback für die History an
    my_callbacks = [
        tf.keras.callbacks.CSVLogger((destpath_logsave + "Training_History.csv"), separator=';', append=False)
    ]

    # Nullt die Variable für die Größe des Modells
    modelsize = 0
    # Speichert die Anzahl der Modell-Parameter in Variablen für die finale Evaluation
    model_trainable_parameter_count = count_params(material_model.trainable_weights)
    model_non_trainable_parameter_count = count_params(material_model.non_trainable_weights)
    model_parameter_total_count = model_trainable_parameter_count + model_non_trainable_parameter_count


    ##################################################################################
    # Printen des Modells
    ################################################################################## 
    print()
    print()
    print("Printen des Modells:")
    print("####################")
    print()
    print(material_model.summary ())
    print()


    ##################################################################################
    # Training des Modells
    ################################################################################## 
    print()
    print("Trainieren des Modells:")
    print("#######################")
    print()
    training_material = material_model.fit(x=x_train_material, y=y_train_material, 
    validation_data=(x_val_material, y_val_material), batch_size=32, shuffle=True, 
    epochs=epochs_train, callbacks=my_callbacks)
    print()


    ##################################################################################
    # Ausgabe auf der Konsole
    ################################################################################## 
    print()
    print("History und Logfiles:")
    print("#####################")


    ##################################################################################
    # Finales speichern des Modells
    ################################################################################## 
    # Speichert die Modelle in der definierten Ordnerstuktur ab
    if saving_model_to_dir == 1:
        destpath_model = destpath_modelsave + "material_model_final.h5"
        material_model.save(destpath_model)
        modelsize = os.path.getsize(destpath_model)
        modelsize = modelsize / 1048576
        modelsize = round(modelsize, 2)
        print()
        print("- Final trainiertes Modell wurde gespeichert")


    ##################################################################################
    # Plotten und Speichern der Trainings-History des Modells
    ################################################################################## 

    plt.plot(training_material.history['mae'])
    plt.plot(training_material.history['val_mae'])
    plt.title('Abweichung')
    plt.ylabel('MAE')
    plt.xlabel('Epoche')
    plt.legend(['Training', 'Validierung'], loc='upper right')
    plt.savefig(destpath_plotsave +"Training"+str(training_number)+ '_mae_vs_val_mae.png')
    plt.savefig(destpath_batchplots +"Training"+str(training_number)+ '_mae_vs_val_mae.png')
    #plt.show()
    plt.clf()

    plt.plot(training_material.history['loss'])
    plt.plot(training_material.history['val_loss'])
    plt.title('Quadratische Abweichung')
    plt.ylabel('MSE')
    plt.xlabel('Epoche')
    plt.legend(['Training', 'Validierung'], loc='upper right')
    plt.savefig(destpath_plotsave +"Training"+str(training_number)+ '_mse_vs_val_mse.png')
    plt.savefig(destpath_batchplots +"Training"+str(training_number)+ '_mse_vs_val_mse.png')
    #plt.show()
    plt.clf()
    
    print("- Plots des Modells wurden gespeichert")

    ##################################################################################
    # Evaluieren der einzelnen Bilder des Trainingsplit per predict
    ##################################################################################
    # Für die Kontrolle einzelner Ergebnisse, werden alle Bilder des Trainingsplits
    # einzeln per predict Befehl evaluiert und in eine CSV exportiert.
    i = 0
    evaluation_train = []
    while i < len(x_train_material):

      x = x_train_material[i]
      x = tf.expand_dims(x,axis=0)
      actual_need = y_train_material[i]
      result = material_model.predict(x)
      result = round(result[0][0])
      deviation = abs(actual_need-result)

      row = []
      row.append(images_train_filenames[i])
      row.append(actual_need)
      row.append(result)
      row.append(deviation)

      evaluation_train.append(row)
      i += 1

    evaluation_train = sorted(evaluation_train, key=lambda x:x[3])

    with open(destpath_logsave + "Evaluation_Train.csv", 'w', newline='', encoding='utf-8-sig') as f_object:  
        writer_object = writer(f_object, delimiter=";")
        writer_object.writerow(["Dateiname", "Tatsächlicher Materialbedarf in g", "geschätzer Materialbedarf in g", "Abweichung Materialbedarf in g"])
        writer_object.writerows(evaluation_train)  
        f_object.close()

    print("- Evaluieren der einzelnen Bilder des Trainingsplit per predict wurde durchgeführt")



    ##################################################################################
    # Evaluieren der einzelnen Bilder des Validationsplit per predict
    ##################################################################################
    # Für die Kontrolle einzelner Ergebnisse, werden alle Bilder des Validationsplits
    # einzeln per predict Befehl evaluiert und in eine CSV exportiert.
    i = 0
    evaluation_val = []
    while i < len(x_val_material):

      x = x_val_material[i]
      x = tf.expand_dims(x,axis=0)
      actual_need = y_val_material[i]
      result = material_model.predict(x)
      result = round(result[0][0])
      deviation = abs(actual_need-result)

      row = []
      row.append(images_val_filenames[i])
      row.append(actual_need)
      row.append(result)
      row.append(deviation)

      evaluation_val.append(row)
      i += 1

    evaluation_val = sorted(evaluation_val, key=lambda x:x[3])

    with open(destpath_logsave + "Evaluation_Validation.csv", 'w', newline='', encoding='utf-8-sig') as f_object:  
        writer_object = writer(f_object, delimiter=";")
        writer_object.writerow(["Dateiname", "Tatsächlicher Materialbedarf in g", "geschätzer Materialbedarf in g", "Abweichung Materialbedarf in g"])
        writer_object.writerows(evaluation_val)  
        f_object.close()


    print("- Evaluieren der einzelnen Bilder des Validationsplit per predict wurde durchgeführt")



    ##################################################################################
    # Evaluieren des Modells 
    ##################################################################################
    # Erzeugt einen Eintrag mit den Trainingsparametern und -ergebnissen in der 
    # Evaluation_Output.csv

    y_train_material_mae = round(training_material.history['mae'][-1])
    y_train_material_share = round((y_train_material_mae / y_train_material_mean), 3)
    y_val_material_mae = round(training_material.history['val_mae'][-1])
    y_val_material_share = round((y_val_material_mae / y_val_material_mean), 3)
    train_val_share = round((y_val_material_share - y_train_material_share), 3)
    model_numbername = "Modell " + str(model_number)
    trainingcount = "Training " + str(training_number)
    images_format = str(widthquer) + "x" + str(heightquer)  

    # Erzeugt eine Liste mit den Variablen die in die Evaluation_Output.csv zurückgeschrieben werden.
    # Eine Beschreibung der Variablen erfolgt weiter unten im Code.
    var_list = [
        date_of_training, 
        time_of_training, 
        model_numbername,
        trainingcount, 
        model_parameter_total_count, 
        modelsize, 
        epochs_train, 
        normalizing_images, 
        standardizing_images, 
        preprocesing_images,
        synthetic_images_mirror,
        synthetic_images_rotation,
        synthetic_images_noise,
        synthetic_images_brightness,
        synthetic_images_sharpness,
        images_with_sticker,
        waende_alle_count,
        waende_train_count,
        waende_val_count, 
        images_all_count, 
        x_train_images_count, 
        x_val_images_count, 
        images_format,
        y_train_material_mean, 
        y_train_material_mae, 
        y_train_material_share, 
        y_val_material_mean, 
        y_val_material_mae, 
        y_val_material_share, 
        train_val_share
    ]

    if os.path.exists(evalpath_output) == False:
      with open(evalpath_output, 'w', newline='', encoding='utf-8-sig') as f_object:
        writer_object = writer(f_object, delimiter=";")
        writer_object.writerow(["Trainingsdatum", 
                                "Trainingszeitpunkt",
                                "Modellnummer",
                                "Trainingsnummer", 
                                "Anzahl der Parameter des Modells", 
                                "Modellgröße in MB", 
                                "Trainierte Epochen",
                                "Normalisierung", 
                                "Standardisierung", 
                                "Preprocessing",
                                "Synthetisch - gespiegelt",
                                "Synthetisch - rotiert",
                                "Synthetisch - Rauschen",
                                "Synthetisch - Helligkeit",
                                "Synthetisch - Schärfe",
                                "Bilder mit Sticker (rot, Aruco)",
                                "Anzahl aller Wände/Szenen", 
                                "Wände Trainingsplit", 
                                "Wände Validationsplit", 
                                "Anzahl aller Bilder", 
                                "Bilder Trainingsplit", 
                                "Bilder Validationsplit", 
                                "Auflösung der Bilder",
                                "Mean Trainingsplit Spachtelmasse", 
                                "Mae Trainingsplit Spachtelmasse", 
                                "Mae/Mean Verhältnis Trainingsplit Spachtelmasse in %", 
                                "Mean Validationsplit Spachtelmasse",
                                "Mae Validationplit Spachtelmasse", 
                                "Mae/Mean Verhältnis Validationsplit Spachtelmasse in %",
                                "Mae Verhältnis Trainings/Validationsplit Spachtelmasse in %" 
                                ])

    with open(evalpath_output, 'a', newline='', encoding='utf-8-sig') as f_object:  
      writer_object = writer(f_object, delimiter=";")
      writer_object.writerow(var_list)  
      f_object.close()


    print("- Allgemeines evaluieren des Modells wurde durchgeführt")
    print()
    print()
    print()
    print()

    #Delete Model / Clear session
    K.clear_session()



Mit dieser Zelle wird ein Teilautomatisiertes Training gestartet. Dabei wird im Vorfeld eine Evaluation_Input.csv eingelesen. Die Evaluation_Input.csv muss mindestens zwei Trainingszeilen enthalten. Über die Evaluation_Input.csv werden einzelne Trainingsparameter übergeben. Zusätzlich können einzelne Parameter in der Zelle definiert werden. Der Code ist entsprechend kommentiert. Nach dem Einlesen der Evaluation_Input.csv wird die Methode *doTrainingwithParameters* für jede Zeile der Datei aufgerufen und ein Training durchgeführt. Nach jedem Training werden Trainingsergebnisse und -paramter an die Evaluation_Output.csv angehängt.

In [ ]:
##################################################################################
# Start Training des Modells (CSV)
##################################################################################

########################################################################
# Hauptverzeichnisse festlegen (Google Colab)
########################################################################
# Lädt Google Drive
from google.colab import drive
drive.mount('/content/drive')


###############################################
# Quellpfade zum Einlesen der Bilder und Labels
###############################################
srcpath_images = "/content/drive/My Drive/Bilddaten"
srcpath_label_infos = "/content/drive/My Drive/Wand_Label_Infos/"
# Wie heißt die CSV Datei in der die Labelinfos gespeichert sind
csv_name = "Wand_Label_Info8.csv"
# Spalte in der CSV Datei in der die Labelinfos gespeichert sind. 
# Achtung counter beginnt mit 0. Für die mitgelieferte Wand_Label_Info8.csv 
# befinden sich die Label für Fugendeckstreifen in Spalte 19 und für
# Spachtelmasse an Stelle 23
csv_spalte_labelinfo = 23

###############################################
# Gibt den Zielpfad für Infos des Trainings an
###############################################
destpath_dir = "/content/drive/My Drive/Training/"

##################################################
# Gibt den Pfad für die Evaluation an
##################################################
# Pfad zur Eingabe-Datei
evalpath_input = "/content/drive/My Drive/Training/Evaluation_Input.csv"
# Pfad zur Ausgabe-Datei an welche die Ergebnisse angehängt werden sollen
evalpath_output = "/content/drive/My Drive/Training/Evaluation_Output.csv"


##################################################################################
# Evaluation_Input_CSV in Array einlesen
##################################################################################
with open(evalpath_input, encoding="utf-8-sig") as evaluation_input_file:
    evaluation_input = np.loadtxt(evaluation_input_file, delimiter=";", skiprows=1, dtype="int")
print(evaluation_input)


i = 0
while i < len(evaluation_input):

    model_number = evaluation_input[i][0]
    # Gibt die Anzahl der Trainingsepochen an
    epochs_train = 100
    # Nummer des Trainings
    # Achtung! Nummer des Trainings nicht wiederverwenden
    training_number = i+1
    # In welchem Verhältnis sollen die Daten in Train-/Validationssplit
    # aufgeteilt werden?
    faktorTrainSplit = 0.8

    images_with_sticker = evaluation_input[i][4]
    # Sollen Bilder welche mit DSLR erstellt wurden verwendet werden?
    images_taken_with_DSLR = 0
    # Sollen Bilder welche mit dem Smartphone erstellt wurden verwendet werden?
    images_taken_with_Smartphone = 1
    # Sollen Querformat-Bilder verwendet werden?
    images_taken_format_landscape = 1
    # Sollen Hochformat-Bilder verwendet werden?
    images_taken_format_portrait = 0

    # Welche Synthetischen Bilder sollen verwendet werden?
    synthetic_images_mirror = evaluation_input[i][7]
    synthetic_images_rotation = evaluation_input[i][8]
    synthetic_images_noise = evaluation_input[i][9]
    synthetic_images_brightness = evaluation_input[i][10]
    synthetic_images_sharpness = evaluation_input[i][11]

    # Sollen die verwendeten Bilder zur Kontrolle in Dateiverzeichnisse geschrieben werden?
    saving_images_to_dir = 0
    # Soll das Modell in Verzeichnisse geschrieben werden?
    saving_model_to_dir = 1
    # Sollen die Bilder Normalisiert werden?
    normalizing_images = evaluation_input[i][1]
    # Sollen die Bilder Standardisiert werden?
    standardizing_images = evaluation_input[i][2]
    # Soll Preprocessing genutzt werden? Dabei werden die Bilder in s/w
    # konvertiert und mit einem Canny Filter die Kanten hervorgehoben
    preprocesing_images = evaluation_input[i][3]

    # Soll die Auflösung der Bilder im Vorfeld angepasst werden?
    resizing_images = 1
    # Werte für die Resize Funktion
    # Zukünftige Maße für Querformat festlegen. Achtung Maße müssen zu Modell passen.
    widthquer = evaluation_input[i][5]
    heightquer = evaluation_input[i][6]
    # Zukünftige Maße für Hochformat festlegen. Achtung Maße müssen zu Modell passen.
    widthhoch = 300
    heighthoch = 400

    doTrainingwithParameters(srcpath_images, srcpath_label_infos, csv_name, csv_spalte_labelinfo, destpath_dir, evalpath_output, model_number, 
                        epochs_train, training_number, images_with_sticker, images_taken_with_DSLR, images_taken_with_Smartphone, 
                        images_taken_format_landscape, images_taken_format_portrait, synthetic_images_mirror, 
                        synthetic_images_rotation, synthetic_images_noise, synthetic_images_brightness, synthetic_images_sharpness, 
                        saving_images_to_dir, saving_model_to_dir, normalizing_images, standardizing_images, preprocesing_images, resizing_images, widthquer, heightquer, faktorTrainSplit)
    i += 1
print()
print("Training durchgeführt")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[[  1   0   0   0   1 200 150   0   0   0   0   0]
 [  1   0   0   0   0 400 300   0   0   0   0   0]]

############################################################################################
#                                                                                          #
#                                  Training 1                                             #
#                                                                                          #
############################################################################################

Parameter:
##########

Modellnummer: 1
Normalisierung: 0
Standardisierung: 0
Preprocessing: 0
Bilder mit Sticker (rot, Aruco, etc.): 1
Auflösung Breite: 200
Auflösung Höhe: 150
Synthetisch - gespiegelt: 0
Synthetisch - rotiert: 0
Synthetisch - Rauschen: 0
Synthetisch - Helligkeit: 0
Synthetisch - Schä

<Figure size 432x288 with 0 Axes>

Mit dieser Zelle wird ein einzelnes Training gestartet. Einzelne Trainingsparamter können in der Zelle definiert werden. Der Code ist entsprechend kommentiert. Nach dem Einlesen der Parameter wird die Methode doTrainingwithParameters aufgerufen und ein Training durchgeführt. Nach dem Training werden Trainingsergebnisse und -paramter an die Evaluation_Output.csv angehängt.

In [ ]:
##################################################################################
# Start Training des Modells (Einzeln)
##################################################################################

########################################################################
# Hauptverzeichnisse festlegen (Google Colab)
########################################################################
# Lädt Google Drive
from google.colab import drive
drive.mount('/content/drive')


###############################################
# Quellpfade zum Einlesen der Bilder und Labels
###############################################
srcpath_images = "/content/drive/My Drive/Bilddaten_mitSynth/"
srcpath_label_infos = "/content/drive/My Drive/Wand_Label_Infos/"
# Wie heißt die CSV Datei in der die Labelinfos gespeichert sind
csv_name = "Wand_Label_Info8.csv"
# Spalte in der CSV Datei in der die Labelinfos gespeichert sind. 
# Achtung counter beginnt mit 0. Für die mitgelieferte Wand_Label_Info8.csv 
# befinden sich die Label für Fugendeckstreifen in Spalte 19 und für
# Spachtelmasse an Stelle 23
csv_spalte_labelinfo = 23

###############################################
# Gibt den Zielpfad für Infos des Trainings an
###############################################
destpath_dir = "/content/drive/My Drive/Training/"

##################################################
# Gibt den Pfad für die Evaluation an
##################################################
# Pfad zur Ausgabe-Datei an welche die Ergebnisse angehängt werden sollen
evalpath_output = "/content/drive/My Drive/Training/Evaluation_Output.csv"



model_number = 1
# Gibt die Anzahl der Trainingsepochen an
epochs_train = 100
# Nummer des Trainings
# Achtung! Nummer des Trainings nicht wiederverwenden
training_number = 1
# In welchem Verhältnis sollen die Daten in Train-/Validationssplit
# aufgeteilt werden?
faktorTrainSplit = 0.8

images_with_sticker = 0
# Sollen Bilder welche mit DSLR erstellt wurden verwendet werden?
images_taken_with_DSLR = 0
# Sollen Bilder welche mit dem Smartphone erstellt wurden verwendet werden?
images_taken_with_Smartphone = 1
# Sollen Querformat-Bilder verwendet werden?
images_taken_format_landscape = 1
# Sollen Hochformat-Bilder verwendet werden?
images_taken_format_portrait = 0

# Welche Synthetischen Bilder sollen verwendet werden?
synthetic_images_mirror = 1
synthetic_images_rotation = 1
synthetic_images_noise = 1
synthetic_images_brightness = 1
synthetic_images_sharpness = 1

# Sollen die verwendeten Bilder zur Kontrolle in Dateiverzeichnisse geschrieben werden?
saving_images_to_dir = 1
# Soll das Modell in Verzeichnisse geschrieben werden?
saving_model_to_dir = 1
# Sollen die Bilder Normalisiert werden?
normalizing_images = 0
# Sollen die Bilder Standardisiert werden?
standardizing_images = 0
# Soll Preprocessing genutzt werden? Dabei werden die Bilder in s/w
# konvertiert und mit einem Canny Filter die Kanten hervorgehoben
preprocesing_images = 0

# Soll die Auflösung der Bilder im Vorfeld angepasst werden?
resizing_images = 1
# Werte für die Resize Funktion
# Zukünftige Maße für Querformat festlegen. Achtung Maße müssen zu Modell passen.
widthquer = 400
heightquer = 300
# Zukünftige Maße für Hochformat festlegen. Achtung Maße müssen zu Modell passen.
widthhoch = 300
heighthoch = 400

doTrainingwithParameters(srcpath_images, srcpath_label_infos, csv_name, csv_spalte_labelinfo, destpath_dir, evalpath_output, model_number, 
                    epochs_train, training_number, images_with_sticker, images_taken_with_DSLR, images_taken_with_Smartphone, 
                    images_taken_format_landscape, images_taken_format_portrait, synthetic_images_mirror, 
                    synthetic_images_rotation, synthetic_images_noise, synthetic_images_brightness, synthetic_images_sharpness, 
                    saving_images_to_dir, saving_model_to_dir, normalizing_images, standardizing_images, preprocesing_images, resizing_images, widthquer, heightquer, faktorTrainSplit)


print()
print("Training durchgeführt")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

############################################################################################
#                                                                                          #
#                                  Training 1                                             #
#                                                                                          #
############################################################################################

Parameter:
##########

Modellnummer: 1
Normalisierung: 0
Standardisierung: 0
Preprocessing: 0
Bilder mit Sticker (rot, Aruco, etc.): 0
Auflösung Breite: 400
Auflösung Höhe: 300
Synthetisch - gespiegelt: 1
Synthetisch - rotiert: 1
Synthetisch - Rauschen: 1
Synthetisch - Helligkeit: 1
Synthetisch - Schärfe: 1


Einlesen der Daten:
###################

Anzahl aller verwendeten Wände/Szenen: 112
Anzahl der